In [52]:
import pandas as pd

image_features=pd.read_csv("..\image_features_mahmood\image_features.csv")

In [53]:
image_features=image_features.drop(image_features.columns[0], axis=1)
image_features.head()

,PetID,Energy,Contrast,Correlation,Entropy,Mean,Std_Dev,num_checkpoints
0,cf8d949f9-2,0.035198,205.034612,0.957454,7.365074,133.539156,52.617205,371
1,61d4dc56b-12,0.037368,96.525711,0.983005,7.061479,88.483483,56.524143,187
2,53923463d-9,0.036733,29.713199,0.995834,6.920583,88.724871,60.340616,251
3,0173c456c-8,0.020710,108.010620,0.984549,7.500592,87.555181,61.033249,722
4,fa7c7d1be-3,0.023931,106.188246,0.982613,7.396937,121.323948,56.922695,486


In [54]:
#clean petID column
image_features["PetID"]=image_features["PetID"].str.split("-", expand=True)[0]

image_features.head(3)

,PetID,Energy,Contrast,Correlation,Entropy,Mean,Std_Dev,num_checkpoints
0,cf8d949f9,0.035198,205.034612,0.957454,7.365074,133.539156,52.617205,371
1,61d4dc56b,0.037368,96.525711,0.983005,7.061479,88.483483,56.524143,187
2,53923463d,0.036733,29.713199,0.995834,6.920583,88.724871,60.340616,251


In [55]:
#Group by ID
image_features_transformed=image_features.groupby("PetID", as_index=False).mean()

In [56]:
sentiment_features=pd.read_csv("..\description_sentiment_features_luis\processed_train.csv")
sentiment_features.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,polarity,subjectivity
0,2,Nibble,3,299,0,1,1,7,0,1,...,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,0.130357,0.407143
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,0.100000,0.400000
2,1,Brisco,1,307,0,1,2,7,0,2,...,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,0.261905,0.542857
3,1,Miko,4,307,0,2,1,2,0,2,...,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,0.341146,0.437500
4,1,Hunter,1,307,0,1,1,0,0,2,...,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,0.400000,0.743750


In [57]:
sentiment_features.shape

(14993, 26)

In [58]:
image_features.shape, image_features_transformed.shape

((72776, 8), (18510, 8))

In [59]:
all_data=sentiment_features.merge(image_features_transformed, how="left", on="PetID")
all_data.head(3)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,AdoptionSpeed,polarity,subjectivity,Energy,Contrast,Correlation,Entropy,Mean,Std_Dev,num_checkpoints
0,2,Nibble,3,299,0,1,1,7,0,1,...,2,0.130357,0.407143,0.035852,157.460689,0.986806,7.433718,95.712315,77.438182,482.000000
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,0,0.100000,0.400000,0.022545,63.340863,0.993091,7.714046,102.676512,68.204360,187.500000
2,1,Brisco,1,307,0,1,2,7,0,2,...,3,0.261905,0.542857,0.030182,78.109415,0.982029,7.185988,142.957730,48.593305,296.857143


In [60]:
all_data.columns , len(all_data.columns)

(Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
        'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
        'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
        'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed',
        'polarity', 'subjectivity', 'Energy', 'Contrast', 'Correlation',
        'Entropy', 'Mean', 'Std_Dev', 'num_checkpoints'],
       dtype='object'),
 33)

In [61]:
columns_to_drop=["Name","Description","RescuerID"]
categorical_columns=["Type",'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health',"State","PetID","AdoptionSpeed"]
numerical_columns=['polarity', 'subjectivity', 'Energy', 'Contrast', 'Correlation',
       'Entropy', 'Mean', 'Std_Dev', 'num_checkpoints','PhotoAmt','VideoAmt',"Age","Quantity","Fee"]


In [62]:
all_data=all_data.fillna(0)

all_data[categorical_columns]=all_data[categorical_columns].astype(str)
all_data[numerical_columns]=all_data[numerical_columns].astype(float)

all_data=all_data.drop(columns_to_drop, axis=1)

all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14993 entries, 0 to 14992
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Type             14993 non-null  object 
 1   Age              14993 non-null  float64
 2   Breed1           14993 non-null  object 
 3   Breed2           14993 non-null  object 
 4   Gender           14993 non-null  object 
 5   Color1           14993 non-null  object 
 6   Color2           14993 non-null  object 
 7   Color3           14993 non-null  object 
 8   MaturitySize     14993 non-null  object 
 9   FurLength        14993 non-null  object 
 10  Vaccinated       14993 non-null  object 
 11  Dewormed         14993 non-null  object 
 12  Sterilized       14993 non-null  object 
 13  Health           14993 non-null  object 
 14  Quantity         14993 non-null  float64
 15  Fee              14993 non-null  float64
 16  State            14993 non-null  object 
 17  VideoAmt    

In [ ]:
import prince

all_data.set_index("AdoptionSpeed")

famd = prince.FAMD(
    n_components=2,
    n_iter=3,
    copy=True,
    check_input=True,
    random_state=42,
    engine="sklearn",
    handle_unknown="error"  # same parameter as sklearn.preprocessing.OneHotEncoder
)
famd = famd.fit(dataset)
